In [4]:
from PIL import Image, ImageDraw
import json

img = Image.open("/mnt/windows/Users/adity/Downloads/output/100_engraved.png").convert("RGB")
draw = ImageDraw.Draw(img)

with open("/mnt/windows/Users/adity/Downloads/output/100.json", "r", encoding="utf-8") as f:
    data = json.load(f)

for item in data:
    x1, y1, x2, y2 = item["bbox"]
    draw.rectangle(
        [x1, y1, x2, y2],
        outline="red",
        width=2
    )

img.save("boxed.png")


In [ ]:
import cv2

img = cv2.imread("0_engraved.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3, 3), 0)
denoise = cv2.medianBlur(gray, 3)

# show denoise:
cv2.imshow("denoise", denoise)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
_, thresh = cv2.threshold(
    denoise, 0, 255,
    cv2.THRESH_BINARY + cv2.THRESH_OTSU
)
cv2.imshow("thresh_otsu", thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
clean = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
cv2.imshow("clean", clean)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 4))
clean_ = cv2.morphologyEx(clean, cv2.MORPH_CLOSE, kernel)
cv2.imshow("clean_", clean_)
cv2.waitKey(0)
cv2.destroyAllWindows()

# saving
cv2.imwrite("0_clean.png", clean_)

True

In [18]:
# doing the above pipeline for each img in "/mnt/windows/Users/adity/Downloads/output/". each image is titled "X_engraved.png" where X is a number from 0 to 999, and saving as "X_clean.png" in the same directory.

import os
for i in range(1000):
    img_path = f"/mnt/windows/Users/adity/Downloads/output/{i}_engraved.png"
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    denoise = cv2.medianBlur(gray, 3)
    _, thresh = cv2.threshold(
        denoise, 0, 255,
        cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    clean = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 4))
    clean_ = cv2.morphologyEx(clean, cv2.MORPH_CLOSE, kernel)
    save_path = f"/mnt/windows/Users/adity/Downloads/output1/{i}_clean.png"
    cv2.imwrite(save_path, clean_)

In [9]:
import json
import os
import cv2
import glob
from sklearn.model_selection import train_test_split
import shutil

# --- Configuration ---
RAW_DATA_DIR = "/mnt/windows/Users/adity/Downloads/output_clean/"
OUTPUT_DIR = "datasets/brahmi_ocr"

def convert_bbox_to_yolo(bbox, img_width, img_height):
    xmin, ymin, xmax, ymax = bbox
    
    dw = 1.0 / img_width
    dh = 1.0 / img_height
    
    w = xmax - xmin
    h = ymax - ymin
    x_center = xmin + w / 2.0
    y_center = ymin + h / 2.0
    
    x_center *= dw
    w *= dw
    y_center *= dh
    h *= dh
    
    return x_center, y_center, w, h

def main():
    # 1. Identify all valid pairs
    print(f"Scanning {RAW_DATA_DIR}...")
    valid_pairs = [] 
    
    for i in range(1000):
        json_filename = f"{i}.json"
        img_filename = f"{i}_clean.png"
        
        json_path = os.path.join(RAW_DATA_DIR, json_filename)
        img_path = os.path.join(RAW_DATA_DIR, img_filename)
        
        if os.path.exists(json_path) and os.path.exists(img_path):
            valid_pairs.append((json_path, img_path, str(i)))
    
    print(f"Found {len(valid_pairs)} valid image-JSON pairs.")

    if len(valid_pairs) == 0:
        print("Error: No data found. Check your paths.")
        return

    # 2. Build Class Vocabulary
    print("Building class vocabulary...")
    unique_chars = set()
    
    for json_path, _, _ in valid_pairs:
        with open(json_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                for item in data:
                    # FIX: Check if 'char' exists before accessing it
                    if 'char' in item:
                        unique_chars.add(item['char'])
            except json.JSONDecodeError:
                print(f"Skipping corrupted JSON: {json_path}")

    char_to_id = {char: idx for idx, char in enumerate(sorted(list(unique_chars)))}
    id_to_char = {idx: char for char, idx in char_to_id.items()}
    
    print(f"Found {len(char_to_id)} unique characters.")
    
    # Save mapping
    with open("class_mapping.json", "w", encoding='utf-8') as f:
        json.dump(char_to_id, f, ensure_ascii=False, indent=2)

    # 3. Prepare Directories
    for split in ['train', 'val']:
        os.makedirs(os.path.join(OUTPUT_DIR, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(OUTPUT_DIR, 'labels', split), exist_ok=True)

    # 4. Split Data
    train_pairs, val_pairs = train_test_split(valid_pairs, test_size=0.2, random_state=42)
    splits = {'train': train_pairs, 'val': val_pairs}

    # 5. Process Files
    print("Converting and moving files...")
    for split, pairs in splits.items():
        for json_path, img_path, base_id in pairs:
            
            img = cv2.imread(img_path)
            if img is None: continue
            height, width = img.shape[:2]

            with open(json_path, 'r', encoding='utf-8') as f:
                annotations = json.load(f)

            yolo_lines = []
            for ann in annotations:
                # FIX: Skip entries without 'char'
                if 'char' not in ann:
                    continue
                
                char = ann['char']
                
                # Verify char is in our list (it should be)
                if char not in char_to_id: 
                    continue
                
                cls_id = char_to_id[char]
                
                # Check if 'bbox' exists too
                if 'bbox' not in ann:
                    continue
                    
                bbox = ann['bbox'] 
                xc, yc, w, h = convert_bbox_to_yolo(bbox, width, height)
                
                if w > 0 and h > 0:
                    yolo_lines.append(f"{cls_id} {xc} {yc} {w} {h}")

            # Only write files if we found valid lines
            if len(yolo_lines) > 0:
                label_out_path = os.path.join(OUTPUT_DIR, 'labels', split, base_id + ".txt")
                with open(label_out_path, 'w', encoding='utf-8') as f:
                    f.write("\n".join(yolo_lines))

                out_img_name = base_id + ".png"
                shutil.copy(img_path, os.path.join(OUTPUT_DIR, 'images', split, out_img_name))

    # 6. Create YAML Config
    yaml_content = f"""
path: {os.path.abspath(OUTPUT_DIR)}
train: images/train
val: images/val

nc: {len(char_to_id)}
names: {list(id_to_char.values())}
    """
    
    with open("brahmi_config.yaml", "w", encoding='utf-8') as f:
        f.write(yaml_content)
        
    print("Data preparation complete. 'brahmi_config.yaml' created.")

if __name__ == "__main__":
    main()

Scanning /mnt/windows/Users/adity/Downloads/output_clean/...
Found 1000 valid image-JSON pairs.
Building class vocabulary...
Found 0 unique characters.
Converting and moving files...
Data preparation complete. 'brahmi_config.yaml' created.
